# Users and User Roles {#sec-users}

::: {.content-hidden when-format="html"}

## Project Setup

Install and load the necessary packages

In [17]:
#| echo: false
#| output: false
import os
os.environ['R_HOME'] = f'C:/Users/{os.environ.get('USERNAME')}/Miniconda3/envs/r_python_jl/Lib/R'

In [18]:
#| echo: false
#| output: false
%load_ext rpy2.ipython
# only have to run once to allow the R magic command

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


::: {.panel-tabset}

#### R

In [19]:
%%capture 
%%R

library("dplyr")
library("jsonlite")
library("tidyr")
library("REDCapR")
library("knitr")
library("remotes")
library("gt")

In [20]:
%%capture --no-display --no-stdout
%%R

version <- packageVersion("REDCapR")
version

[1] '1.1.9005'


In this project, we will use the bleeding edge version of REDCapR available on Github

In [6]:
%%capture --no-display --no-stdout
%%R

# Detach REDCapR if already loaded, and download the latest version
if (version!='1.1.9005') {
    detach("package:REDCapR", unload=TRUE)
    remotes::install_github("OuhscBbmc/REDCapR")
    library("REDCapR")
    print(packageVersion("REDCapR"))
}

else {
    print("REDCapR package up to date")
}

[1] "REDCapR package up to date"


#### Python

In [21]:
import redcap
import json
import pandas as pd

:::

Assign your project URL and Token

::: {.panel-tabset}

#### R

In [22]:
%%R
path = paste0("C:/Users/", Sys.getenv("USERNAME"), '/json_api_data.json')
token <- jsonlite::fromJSON(path)$dev_token$'308'
url <- "https://dev-redcap.doh.wa.gov/api/"

#### Python

In [23]:
path_to_json = f"C:/Users/{os.environ.get('USERNAME')}/json_api_data.json"
api_key = json.load(open(path_to_json))
api_token = api_key['dev_token']['308']
api_url = api_key['dev_url']
project = redcap.Project(api_url, api_token)

:::

:::

REDCap allows strict control of user rights for each project. These rights include a user's ability to edit, export and view data, add or edit reports, import data, create records, etc. Privileges to view and export data are specified for each instrument. 

User roles can be set with predefined privileges and users can then be assigned to these user role groups. 

Both REDCapR and Pycap have functions for exporting user rights.

::: {.panel-tabset}

#### R

`redcap_users_export()`

In [12]:
%%capture --no-display --no-stdout
%%R
users_data <- redcap_users_export(redcap_uri = url, token = token)
names(users_data)

[1] "data_user"       "data_user_form"  "success"         "status_code"    
[5] "outcome_message" "elapsed_seconds" "raw_text"       


**Extract the `data_user` table to view who has access to the REDCap project:**

`redcap_users_export` output provides a list with several elements. The two most useful elements are the `data_user` and the `data_user_form`. 

In [13]:
%%capture
%%R
users <- users_data$data_user

::: {.content-hidden when-format="html"}

In [14]:
%%R
users_tbl <- gt(head(users))
gt::gtsave(users_tbl, filename = 'export_users1.html', path = "./files/export_files/")

:::

<p align="center"><iframe width="100%" height="200" src="./files/export_files/export_users1.html" title="Quarto Documentation"></iframe></p>

There is no way to export which user is in which named user role group with REDCapR or the native API. When you export this data, if a user has been assigned to a user role group, then it will return the user with the role's defined privileges.

**Extract the `data_user_form` table to view which forms each users has access to:**

In [28]:
%%capture
%%R
users_forms <- users_data$data_user_form

::: {.content-hidden when-format="html"}

In [29]:
%%R
users_forms_tbl <- gt(users_forms)
gt::gtsave(users_forms_tbl, filename = 'export_users2.html', path = "./files/export_files/")

:::

<p align="center"><iframe width="100%" height="200" src="./files/export_files/export_users2.html" title="Quarto Documentation"></iframe></p>

#### Python

`export_users()`, `export_user_roles()`, `export_user_role_assignment()`

**Use `export_users` to view who has access to the REDCap project and what their rights are:**

In [14]:
project.export_users(format_type='df') 

,username,email,firstname,lastname,expiration,data_access_group,data_access_group_id,design,alerts,user_rights,...,mobile_app,mobile_app_download_data,record_create,record_rename,record_delete,lock_records_all_forms,lock_records,lock_records_customization,forms,forms_export
0,alexey.gilman@doh.wa.gov,Alexey.Gilman@doh.wa.gov,Alexey,Gilman,NaN,records1_2,2716.0,0,0,1,...,1,1,1,1,1,1,0,1,"demographics:3,symptoms:1,test_information:3,c...","demographics:1,symptoms:1,test_information:1,c..."
1,caitlin.drover@doh.wa.gov,Caitlin.Drover@doh.wa.gov,Caitlin,Drover,NaN,NaN,NaN,1,1,1,...,1,0,1,1,1,0,0,1,"demographics:1,symptoms:1,test_information:1,c...","demographics:1,symptoms:1,test_information:1,c..."
2,emily.pearman@doh.wa.gov,emily.pearman@doh.wa.gov,Emily,Pearman,NaN,NaN,NaN,1,1,1,...,1,1,1,1,1,0,0,0,"demographics:1,symptoms:1,test_information:1,c...","demographics:1,symptoms:1,test_information:1,c..."


**Use `export_user_roles` to view the defined user rold groups for that project and their permissions:**

In [15]:
project.export_user_roles(format_type='df') 

,unique_role_name,role_label,design,alerts,user_rights,data_access_groups,reports,stats_and_charts,manage_survey_participants,calendar,...,mobile_app,mobile_app_download_data,record_create,record_rename,record_delete,lock_records_customization,lock_records,lock_records_all_forms,forms,forms_export
0,U-5354FA3HYL,Admin,1,1,1,1,1,1,1,1,...,1,0,1,1,1,1,0,0,"demographics:1,symptoms:1,test_information:1,c...","demographics:1,symptoms:1,test_information:1,c..."


**Use `export_user_role_assignment` to view which user is in in which role group and data access group:**

In [16]:
project.export_user_role_assignment(format_type='df')

,username,unique_role_name,data_access_group
0,alexey.gilman@doh.wa.gov,NaN,records1_2
1,caitlin.drover@doh.wa.gov,U-5354FA3HYL,NaN
2,emily.pearman@doh.wa.gov,NaN,NaN


You can use the `unique_role_name` field from the `export_user_roles` output to see which user is assigned to which user role in the ouput of `export_user_role_assignment`.

:::

The user rights tables also include information on which Data Access Group each user is assigned to. For more information on Data Access Groups see @sec-DAG